In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import sys
sys.path.append("../") # go to parent dir
from misc.helpers import *

height, weight, gender = load_data(sub_sample=False, add_outlier=False)
x, mean_x, std_x = standardize(height)
y, tx = build_model_data(x, weight)

In [3]:
def compute_loss(y, tx, w, type='mse'):

    """Calculate the loss using either MSE or MAE.

    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        w: numpy array of shape=(2,). The vector of model parameters.
        type: string that can take value 'mse' or 'mae'.

    Returns:
        the value of the loss (a scalar), corresponding to the input parameters w.
    """
    # Compute loss by MSE
    error = y - np.dot(tx, w)
    N = len(tx)
    if type == 'mse':
        loss = 1/(2*N)*np.sum(np.square(error))
    elif type == 'mae':
        loss = 1/N*np.sum(np.abs(error))

    return loss

def compute_gradient(y, tx, w):
    """Computes the gradient at w.
        
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        w: numpy array of shape=(2, ). The vector of model parameters.
        
    Returns:
        An numpy array of shape (2, ) (same shape as w), containing the gradient of the loss at w.
    """
    # Compute gradient vector
    error = y - np.dot(tx, w)
    N = len(tx)
    gradient_w0 = -1/N*np.sum(error)
    gradient_w1 = -1/N*np.sum(error*tx[:,1])

    gradient = np.array([gradient_w0, gradient_w1]).T

    return gradient


def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient at w from just few examples n and their corresponding y_n labels.
        
    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        w: numpy array of shape=(2, ). The vector of model parameters.
        
    Returns:
        A numpy array of shape (2, ) (same shape as w), containing the stochastic gradient of the loss at w.
    """
    N = len(tx)
    error = y - np.dot(tx, w)
    stoch_gradient = -1/N*np.dot(tx.T, error)

    return stoch_gradient

# Generate the grid of parameters to be swept
grid_w0, grid_w1 = generate_w(num_intervals=10)
grid_losses = grid_search(y, tx, grid_w0, grid_w1, compute_loss)

In [4]:
def AdaGrad(w_0, tx, y, max_iter, lambda_, epsilon = 1e-8):
        '''Algoritm for adaptive gradient optimization.

        Adapts learing parameter - smaller rate for frequent features (well-suited for sparse data).

        Parameters
        ----------
        w_0 : ndarray of shape (D, 1)
            Initial weights of the model
        tx : ndarray of shape (N, D)
            Array of input features
        y : ndarray of shape (N, 1)
            Array of output
        max_iter : int
            Maximum number of iterations
        Returns
        -------
        grads : ndarray of shape (max_iter, D)
            Array of gradient estimators in each step of the algorithm.
        '''
        D = len(w_0)
        G_t = np.zeros((D, D))

        # Outputs
        grads = []
        w = [w_0]
        losses = []

        for t in range(max_iter):
            g_t = compute_gradient(y, tx, w[t])
            G_t += g_t**2
            v_k = np.diag(lambda_ / np.sqrt(G_t + epsilon)) @ g_t
            w_next = w[t] - v_k
            w.append(w_next)
            grads.append(v_k)

            loss = compute_loss(y, tx, w[t])
            losses.append(loss)

            print("AG iter. {bi}/{ti}: loss={l}, w0={w0}, w1={w1}".format(
              bi=t, ti=max_iter - 1, l=loss, w0=w[t][0], w1=w[t][1]))

        return losses, w

In [14]:
# Define the parameters of the algorithm.
max_iters = 50

# Initialization
w_initial = np.array([0, 0])

# Start AG.
start_time = datetime.datetime.now()
ag_losses, ag_ws = AdaGrad(w_initial, tx, y, max_iters, 1e1)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("AG: execution time={t:.3f} seconds".format(t=exection_time))

# Near optimum: loss=3.0832601417144585, w0=73.10945680053558, w1=11.967378197209419

AG iter. 0/49: loss=2792.2367127591674, w0=0, w1=0
AG iter. 1/49: loss=1456.7640240305902, w0=19.99999999971552, w1=19.99999999971552
AG iter. 2/49: loss=1387.694382451022, w0=21.526497503422412, w1=21.526497503422412
AG iter. 3/49: loss=1377.8409098229004, w0=21.753044884084776, w1=21.753044884084776
AG iter. 4/49: loss=1375.6019571486906, w0=21.804853620553107, w1=21.804853620553107
AG iter. 5/49: loss=1374.9427088473406, w0=21.820132159322355, w1=21.820132159322355
AG iter. 6/49: loss=1374.7130239259768, w0=21.82545781051292, w1=21.82545781051292
AG iter. 7/49: loss=1374.622839817151, w0=21.827549248284324, w1=21.827549248284324
AG iter. 8/49: loss=1374.5840937154567, w0=21.82844786203259, w1=21.82844786203259
AG iter. 9/49: loss=1374.5662282819485, w0=21.828862216311954, w1=21.828862216311954
AG iter. 10/49: loss=1374.557505946151, w0=21.829064516964355, w1=21.829064516964355
AG iter. 11/49: loss=1374.5530408523907, w0=21.82916807841965, w1=21.82916807841965
AG iter. 12/49: loss=13

In [26]:
def SpiderBoost(w_0, tx, y, max_iter):
        """Algorithm for gradient optimization, which estimates gradients and reduces their iterative variance.

        Parameters
        ----------
        w_0 : ndarray of shape (D, 1)
            Initial weights of the model
        tx : ndarray of shape (N, D)
            Array of input features
        y : ndarray of shape (N, 1)
            Array of output
        max_iter : int
            Maximum number of iterations
        Returns
        -------
        grads : ndarray of shape (max_iter, D)
            Array of gradient estimators in each step of the algorithm.
        """
        # Intrinsic parameters initialization
        N = len(tx)
        n = len(y)
        lipshitz_const = np.linalg.norm(tx, ord='fro')**2

        # Outputs
        grads = []
        w = [w_0]
        losses = []

        # Algorithm
        for t in range(max_iter):
            if t % n == 0:
                v_k = compute_gradient(y, tx, w[t]) # logistic cost function full gradient
            else:
                i = np.random.choice(np.arange(1, n))
                v_k = partial_sum = compute_stoch_gradient(y[i], tx[i], w[t]) - compute_stoch_gradient(y[i], tx[i], w[t-1]) + v_k
            w_next = w[t] - 1/(2*lipshitz_const)*v_k
            w.append(w_next)
            grads.append(v_k)

            loss = compute_loss(y, tx, w[t])
            losses.append(loss)

            print("SB iter. {bi}/{ti}: loss={l}, w0={w0}, w1={w1}".format(
              bi=t, ti=max_iter - 1, l=loss, w0=w[t][0], w1=w[t][1]))

        return losses, w


In [27]:
# Define the parameters of the algorithm.
max_iters = 50

# Initialization
w_initial = np.array([0, 0])

# Start SB.
start_time = datetime.datetime.now()
sb_losses, sb_ws = SpiderBoost(w_initial, tx, y, max_iters)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SB: execution time={t:.3f} seconds".format(t=exection_time))

SB iter. 0/49: loss=2792.2367127591674, w0=0, w1=0
SB iter. 1/49: loss=2792.097871953454, w0=0.0018323480500526278, w1=0.00033699281087472396
SB iter. 2/49: loss=2791.9590363295856, w0=0.0036646729777044798, w1=0.0006739844248472487
SB iter. 3/49: loss=2791.8202057745266, w0=0.005496975559131376, w1=0.0010109789980727712
SB iter. 4/49: loss=2791.6813807190574, w0=0.007329252960889827, w1=0.0013479599672669162
SB iter. 5/49: loss=2791.542560435041, w0=0.009161510201058645, w1=0.001684954060197257
SB iter. 6/49: loss=2791.40374522717, w0=0.010993745041640435, w1=0.0020219508311733227
SB iter. 7/49: loss=2791.2649362745337, w0=0.012825950381452754, w1=0.0023589013960855754
SB iter. 8/49: loss=2791.1261318436464, w0=0.014658137595163353, w1=0.00269587251681817
SB iter. 9/49: loss=2790.9873328850827, w0=0.01649029979548199, w1=0.00303283110126795
SB iter. 10/49: loss=2790.8485390777146, w0=0.018322439075520773, w1=0.0033697895863941947
SB iter. 11/49: loss=2790.7097510597487, w0=0.020154551